In [1]:
import os
import gzip
import pickle
import requests
import subprocess
import statistics
from collections import defaultdict

import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
pd.reset_option('all')


: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



/home/zpimentel/miniconda3/envs/VirTect/lib/python3.6/site-packages/pandas/_config/config.py:622: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


Create necessary directories and download RefSeq assembly summaries and NCBI taxonomy data.

In [3]:
if not os.path.exists('./genomes'):
    os.makedirs('./genomes')

def download_file(url, outfile):
    r = requests.get(url, allow_redirects=True)
    open(outfile, 'wb').write(r.content)

def unzip_db(filename, outdir):
    cmd = ['unzip', filename, '-d', outdir]
    unzip_call = subprocess.call(cmd, shell=False, stderr=subprocess.STDOUT)

refseq_handle = "https://ftp.ncbi.nih.gov/genomes/refseq/assembly_summary_refseq.txt"
taxdmp = 'https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdmp.zip'

download_file(refseq_handle, 'assembly_summary_refseq.txt')
download_file(taxdmp, 'taxdmp.zip')
unzip_db('taxdmp.zip', 'taxonomy')

df = pd.read_csv(refseq_handle, skiprows=1, sep="\t")

/home/zpimentel/miniconda3/envs/VirTect/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Parse nodes.dmp from NCBI taxonomy (map each taxid to its parent taxid) and build a function that will be used to classify every genome from the RefSeq assembly summary as eukaryotic, bacteria, archaea, or virus based on taxid.

In [4]:
node_dict = {}
nodes_file = open("./taxonomy/nodes.dmp")
for row in nodes_file:
    row = row.strip().split('\t|\t')
    child = row[0]
    parent = row[1]
    node_dict[child] = parent
    

def assign_genomes(taxid, node_dict = node_dict):
    """
    Given a NCBI taxonomy ID, this function figures out the domain that ID belongs to.
    """
    eukaryote = 2759
    bacteria = 2
    archaea = 2157
    virus = 10239
    
    taxa_list = [eukaryote, bacteria, archaea, virus]
    classification = "not classified"
        
    while taxid not in taxa_list:
        try:
            taxid = int(node_dict[str(taxid)])
            return(assign_genomes(taxid, node_dict)) # get the parent taxid and try again
        except:
            return(classification)

    if taxid == eukaryote: classification = 'eukaryote'
    if taxid == bacteria: classification = 'bacteria'
    if taxid == archaea: classification = 'archaea'
    if taxid == virus: classification = 'virus'
    
    return(classification)

Assign every genome as eukaryotic, bacteria, archaea, or virus.

In [5]:
df['Classification'] = df['taxid'].apply(assign_genomes)
df.to_csv("assembly_summary_refseq_labeled.csv")

Download 10 randomly sampled eukaryotic genomes and 50 randomly sampled bacterial genomes for training. Only trying 10 euks because I am crazy and doing this on my laptop. This takes a while =(

In [6]:
sampled_euks = df[df['Classification'] == 'eukaryote'].sample(50, random_state=234)
sampled_bact = df[df['Classification'] == 'bacteria'].sample(50, random_state=345)

sampled_euks[sampled_euks.columns[0]] = sampled_euks[sampled_euks.columns[0]].str.split('.').str[0]
sampled_bact[sampled_bact.columns[0]] = sampled_bact[sampled_bact.columns[0]].str.split('.').str[0]

sampled_euks_list = sampled_euks[sampled_euks.columns[0]].to_list()
sampled_bact_list = sampled_bact[sampled_bact.columns[0]].to_list()

for ftp in sampled_euks['ftp_path'].to_list():
    genome_id = ftp.split("/")[-1]
    gen_url = os.path.join(ftp,genome_id + "_genomic.fna.gz").replace(" ", "_")
    r = requests.get(gen_url, allow_redirects=True)
    open("genomes/" + genome_id + ".fna.gzip", 'wb').write(r.content)

for ftp in sampled_bact['ftp_path'].to_list():
    genome_id = ftp.split("/")[-1]
    gen_url = os.path.join(ftp,genome_id + "_genomic.fna.gz").replace(" ", "_")
    r = requests.get(gen_url, allow_redirects=True)
    open("genomes/" + genome_id + ".fna.gzip", 'wb').write(r.content)

For each genome, up to ~1 Mb is used and is broken into non-overlapping contigs of ~50 Kb which were used to count the frequencies of canonical 5-mers computed with a sliding window (meaning each 5-mer and its reverse complement are considered the same). To reduce memory requirements and runtime, I decided to use canonical kmers, used fairly large contigs (50 Kb), and only used a portion of the eukaryotic genomes (up to 1 Mb). I use the ~ because I could be much more stringent about actually making sure everything is the same size - contigs generated from the ends of a chromosome/contig could be smaller for example. I do some QC to remove small stuff later on though. This also takes a while =(

In [7]:
def is_valid_sequence(seq):
    ''' Used to remove any kmers with N's in them '''
    return set(seq).issubset({"A", "T", "C", "G"})

kmers_visited = set()
kmer_dict = defaultdict(lambda:0)
contigs = set(" ")

directory = "./genomes"
euk_count = 0
bact_count = 0
for filename in os.listdir(directory):
    if filename.endswith(".fna.gzip"):
        genome_bp_count = 0
        
        genome_id = "_".join(filename.split("_")[0:2]).split('.')[0]
        
        if genome_id in sampled_euks_list: genome_type = 'euk'
        elif genome_id in sampled_bact_list: genome_type = 'bact'
        else: genome_type = 'NA'
            
        #print(genome_id, genome_type)
        
        with gzip.open(os.path.join(directory, filename), "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                if genome_bp_count < 1000000:
                    if len(record.seq) >= 50000:
                        for i in range(0, len(record.seq), 50000):
                            genome_bp_count += 50000
                            if genome_type == 'euk': 
                                euk_count += 1
                                contig_5kb_name = "euk_" + str(euk_count)
                            elif genome_type == 'bact': 
                                bact_count += 1
                                contig_5kb_name = "bact_" + str(bact_count)
                            contigs.add(contig_5kb_name)
                            
                            if genome_bp_count < 1000000:
                                contig_5kb = record.seq[i:i+50000] 
                                for j in range(len(contig_5kb)):
                                    kmer = contig_5kb[j:j+5].upper()
                                    if kmer.reverse_complement() in kmers_visited:
                                        kmer = kmer.reverse_complement()
                                    if len(kmer) == 5:
                                        if is_valid_sequence(kmer) is True:
                                            kmers_visited.add(str(kmer))
                                            kmer_dict[(contig_5kb_name, str(kmer))] += 1
                else:
                    continue


/home/zpimentel/miniconda3/envs/VirTect/lib/python3.6/site-packages/Bio/Seq.py:155: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


Export the counts of the 512 unique 5-mers per contig in a table so that I can jump back in here without having to re-download the genomes or count the kmers again.

In [8]:
f = open("kmer_matrix.tsv", "w")

f.write("Contig" + "\t")
f.write("\t".join(contigs))
f.write("\n")
for kmer in kmers_visited:
    f.write(kmer + "\t")
    for contig in contigs:
        f.write(str(kmer_dict[(contig, kmer)]) + "\t")
    f.write("\n")
    
f.close()

Load the 5-mer count data per genome into a pandas dataframe, remove chunks with less than 10,000 kmers, and transform the counts to proportions by dividing each kmer count per genome by the total number of kmers per contig.

In [28]:
kmer_df = pd.read_csv("kmer_matrix.tsv", sep="\t", index_col=False)
kmer_df = kmer_df.set_index("Contig")
kmer_df = kmer_df.loc[:, kmer_df.sum(axis=0) > 5000]
kmer_df = kmer_df.div(kmer_df.sum(axis=0),axis=1)
kmer_df = kmer_df.reindex(sorted(kmers_visited))

Prepare data for training. Make a list of lists where each sublist contains the kmer frequncies from one column/contig. Make another list containing the dummy variable for each list (containing the kmer frequencies per contig) whether it be eukaryotic (1) or bacterial (0).

In [30]:
contig_kmers = []
for column in kmer_df:
    kmer_list = kmer_df[column].tolist()
    contig_kmers.append(kmer_list)
    
prelabels = kmer_df.columns.tolist()
labels = []
for i in prelabels:
    if i.split("_")[0] == "euk":
        labels.append(1)
    else:
        labels.append(0)
        
with open('kmer_list.pkl', 'wb') as f:
    pickle.dump(kmers_visited, f)

Split the data into training and testing sets.

In [31]:
X_train, X_test, y_train, y_test = train_test_split(contig_kmers, labels, test_size=0.3, random_state=42)

Train the SVM and save it.

In [32]:
clf = svm.SVC()
clf.fit(X_train, y_train)

model_outfile = 'kmer_SVM.sav'
pickle.dump(clf, open(model_outfile, 'wb'))

Assess the accuracy of the model using the testing data. Need to remember that although these test contigs were not included in the training data, they are from the same genomes used in the training set. Therefore, additional validation is needed to assess accuracy on genomes the model has not seen.

In [33]:
predictions = clf.predict(X_test)
print(f"Accuracy in the testing set is {accuracy_score(y_test, predictions) * 100}%")

Accuracy in the testing set is 99.56222639149468%


Things to improve: Rather than using randomly selected eukaryotic and bacterial genomes for training select a set of taxonomically diverse genomes, make more efforts to make sure there is an even amount of bacterial and euk data in the training data, include a separate validation set of unseen genomes. Specifically, I know the model performs poorly on GC-rich bacterial genomes which weren't seen in the training set, so inclusion of some of those could help.